In [ ]:
import os, random, shutil
from pathlib import Path

In [ ]:
!pip install mtcnn opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from mtcnn import MTCNN
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content


drive  sample_data


In [ ]:
!ls /content/drive


MyDrive  Othercomputers


In [ ]:
!ls "/content/drive/MyDrive/AI_Project"


aligned_224  archive  Final_data_raw


In [ ]:
from pathlib import Path
import cv2
from mtcnn import MTCNN

RAW_DIR = Path("/content/drive/MyDrive/AI_Project/Final_data_raw/Final_data_raw")
ALIGNED_DIR = Path("/content/drive/MyDrive/AI_Project/aligned_224*224")

ALIGNED_DIR.mkdir(parents=True, exist_ok=True)

detector = MTCNN()
IMG_SIZE = 224
MAX_CLASSES = 50
MAX_IMAGES_PER_CLASS = 50

#  اختيار 50 كلاس بس
persons = [p for p in RAW_DIR.iterdir() if p.is_dir()]
persons = persons[:MAX_CLASSES]

for person_folder in persons:
    aligned_person_dir = ALIGNED_DIR / person_folder.name
    aligned_person_dir.mkdir(parents=True, exist_ok=True)

    #  50 صورة بس لكل شخص
    images = list(person_folder.glob("*"))[:MAX_IMAGES_PER_CLASS]

    for img_path in images:
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(img_rgb)
        if len(faces) == 0:
            continue

        face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
        x, y, w, h = face['box']
        x, y = max(0, x), max(0, y)

        face_img = img[y:y+h, x:x+w]
        if face_img.size == 0:
            continue

        face_img = cv2.resize(face_img, (IMG_SIZE, IMG_SIZE))
        cv2.imwrite(str(aligned_person_dir / img_path.name), face_img)

print("✅ Alignment done: 50 classes × up to 50 images each")


✅ Alignment done: 50 classes × up to 50 images each


In [ ]:
from pathlib import Path

ALIGNED_DIR = Path("/content/drive/MyDrive/AI_Project/aligned_224*224")

persons = [p for p in ALIGNED_DIR.iterdir() if p.is_dir()]
print("عدد الأشخاص اللي اتعملهم alignment:", len(persons))

# نشوف مثال
for p in persons[:50]:
    imgs = list(p.glob("*"))
    print(p.name, "→", len(imgs), "صور")


عدد الأشخاص اللي اتعملهم alignment: 50
n000001 → 46 صور
n000004 → 50 صور
n000009 → 48 صور
n000010 → 50 صور
n000008 → 50 صور
n000006 → 50 صور
n000026 → 50 صور
n000028 → 49 صور
n000030 → 50 صور
n000029 → 50 صور
n000037 → 50 صور
n000040 → 50 صور
n000043 → 50 صور
n000047 → 50 صور
n000038 → 49 صور
n000052 → 50 صور
n000054 → 50 صور
n000056 → 50 صور
n000065 → 49 صور
n000067 → 50 صور
n000077 → 50 صور
n000071 → 50 صور
n000069 → 50 صور
n000082 → 50 صور
n000079 → 50 صور
n000083 → 50 صور
n000090 → 50 صور
n000093 → 50 صور
n000091 → 49 صور
n000092 → 49 صور
n000094 → 50 صور
n000103 → 50 صور
n000106 → 50 صور
n000114 → 50 صور
n000116 → 50 صور
n000115 → 50 صور
n000107 → 50 صور
n000117 → 50 صور
n000118 → 50 صور
n000120 → 49 صور
n000122 → 50 صور
n000141 → 50 صور
n000148 → 50 صور
n000149 → 50 صور
n000150 → 50 صور
n000151 → 50 صور
n000160 → 47 صور
n000167 → 50 صور
n000169 → 50 صور
n000171 → 50 صور


In [ ]:
import random, shutil
from pathlib import Path

random.seed(42)

raw_dir = Path("/content/drive/MyDrive/AI_Project/aligned_224*224")
subset_dir = Path("/content/drive/MyDrive/AI_Project/Final_data_split")
train_dir = subset_dir / "train"
val_dir   = subset_dir / "val"

for d in [subset_dir, train_dir, val_dir]:
    d.mkdir(parents=True, exist_ok=True)

all_persons = [p for p in raw_dir.iterdir() if p.is_dir()]
print("عدد الأشخاص المتاحين:", len(all_persons))

selected_persons = all_persons  # نشتغل على الموجود حاليًا

val_ratio = 0.2
MIN_IMAGES = 5

for person_folder in selected_persons:
    images = list(person_folder.glob("*"))

    if len(images) < MIN_IMAGES:
        continue

    random.shuffle(images)
    n_val = max(1, int(len(images) * val_ratio))

    val_images = images[:n_val]
    train_images = images[n_val:]

    train_person = train_dir / person_folder.name
    val_person   = val_dir / person_folder.name
    train_person.mkdir(exist_ok=True)
    val_person.mkdir(exist_ok=True)

    for img in train_images:
        shutil.copy2(img, train_person / img.name)

    for img in val_images:
        shutil.copy2(img, val_person / img.name)

print("✅ Split completed using partial aligned data")


عدد الأشخاص المتاحين: 50
✅ Split completed using partial aligned data


In [ ]:
for person_folder in selected_persons:
    images = list(person_folder.glob("*"))

    MIN_IMAGES = 5   # أقل عدد صور نقبله
    if len(images) < MIN_IMAGES:
        continue

    random.shuffle(images)

    val_ratio = 0.2
    n_val = max(1, int(len(images) * val_ratio))

    val_images = images[:n_val]
    train_images = images[n_val:]

    # إنشاء فولدرات الشخص
    train_person = train_dir / person_folder.name
    val_person   = val_dir / person_folder.name
    train_person.mkdir(exist_ok=True)
    val_person.mkdir(exist_ok=True)

    for img in train_images:
        shutil.copy2(img, train_person / img.name)

    for img in val_images:
        shutil.copy2(img, val_person / img.name)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
SEED = 42

train_dir = Path("/content/drive/MyDrive/AI_Project/Final_data_split/train")
val_dir   = Path("/content/drive/MyDrive/AI_Project/Final_data_split/val")

classes = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
NUM_CLASSES = len(classes)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    zoom_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=classes,
    class_mode="categorical",
    shuffle=True,
    seed=SEED
)

Found 2392 images belonging to 50 classes.
